<div style="direction:rtl;line-height:300%;">
<font face="XB Zar" size=5>
<div align=center>
<font face="B Titr" size=5>
<p></p><p></p>
بسمه تعالی
<p></p>
</font>
<p></p>
<font>
<br>
درس بازیابی پیشرفته اطلاعات
<br>
مدرس: دکتر بیگی
</font>
<p></p>
<br>
<font>
<b>تمرین دوم</b>
</font>
<br>
<br>
موعد تحویل: ... آبان <br>
<br>
<br>
<br>
<font>
دانشگاه صنعتی شریف
<br>
دانشکده مهندسی کامپیوتر
<br>
<br>
</font>
</div>
</font>
</div>

mobina salimipanah - 99109788

<div dir='rtl'>
<h1>
مقدمه
</h1>
<p>
در این تمرین قصد داریم به مباحث زیر بپردازیم:
    <li>مدل‌های برداری</li>
    <li>امتیازدهی و ارزیابی سیستم بازیابی</li>
    <li>مدل‌های احتمالاتی</li>

دیتاست مورد استفاده در این تمرین را می‌توانید در کنار این فایل مشاهده کنید. همچنین لطفا پس از اتمام تمرین یک بار از اول تا آخر نوت‌بوک را اجرا کنید تا مطمئن باشید تمام سل‌ها به درستی کار می‌کنند.

کتابخانه‌های مورد نظرتان را هم می‌توانید در اولین سل نوت‌بوک فراخوانی کنید.
</p>
</div>

<div dir='rtl'>
<h1>1.
آماده‌سازی دیتاست
</h1>
<p>
با استفاده از تمرین قبل و عملیات‌هایی که در آنجا برای پیش‌پردازش متون پیاده‌سازی کردید، دیتاست داده شده را لود کنید. قرار است در ادامه با این دیتاست بخش‌های بعدی تمرین را پیاده‌سازی کنید. همچنین در صورتی که اجرای سل‌های بعدی برایتان طول کشید، می‌توانید آن را کوچک کنید. مثلا از ۷۰درصد دیتای آن استفاده کنید.
</p>
</div>

In [6]:
import math
import pandas as pd
import numpy as np
from collections import defaultdict
from typing import List, Union

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
import csv

In [7]:
# load the dataset
data = []
with open('./AI & Bioinformatics/data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append((row['paperId'], row['title'], row['abstract']))


In [8]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mobina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mobina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mobina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
# preprocess
class Preprocessor:

    def __init__(self, stopwords_path = None):
      # Create a variable of stop words.

        if stopwords_path:
            with open(stopwords_path, 'r') as file:
                for f in file:
                    line = ""
                    for char in f:
                        if char.strip():
                            line += char
                    self.stopwords.append(line)
        else:
            self.stopwords = set(stopwords.words('english'))

    def preprocess(self, text):
      # The main function of the class.
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        # Normalize text (lower case, stemming, lemmatization, etc.)
        text = text.lower()
        words = self.word_tokenize(text)

        #stemming
        stemmed_words = []
        for w in words:
            stemmed_words.append(PorterStemmer().stem(w))
        words = stemmed_words

        #lemmatizing
        words = [WordNetLemmatizer().lemmatize(w) for w in words]

        return ' '.join(words)


    def remove_links(self, text):
        # Remove links
        return re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)

    def remove_punctuations(self, text):
        # Remove punctutaions
        return re.sub(r'[^\w\s]','',text)

    def word_tokenize(self, text):
        # Tokenize text
        return nltk.word_tokenize(text)

    def remove_stopwords(self, words):
        # Remove stopwords
        not_stopwords = []
        for word in words:
            if word not in self.stopwords:
                not_stopwords.append(word)
        return not_stopwords



<div dir='rtl'>
<h2>1-1.
نمایه‌سازی
</h2>
<p>
در این بخش باید برای سامانه یک
Positional Index
بسازید.
<br>
با توجه به مواردی که در بخش بعد می‌آید و نیاز به جست‌وجو‌ی مجزا و با امتیازدهی متفاوت بر روی بخش‌های مختلف سند مثل عنوان یا چکیده آن، در این قسمت باید نمایه‌ی مناسب برای امکان جست‌وجو‌ در بخش‌های مختلف را پیاده‌سازی کنید.
با استفاده از نمایه‌ی ساخته‌شده باید بتوان شماره‌ی تمامی اسنادی که یک کلمه در آن آمده است و همچنین همه‌ی جایگاه‌های این کلمه در هر بخش از هر سند را پیدا کرد.

برای این بخش می‌توانید از نمایه‌ای که در تمرین اول زدید استفاده کنید.
</p>
</div>

In [10]:
#process data
preprocessor = Preprocessor()
corpus = {}
for row in data:
  paperId, title, abstract = row
  processed_title = preprocessor.preprocess(title)
  processed_abstract = preprocessor.preprocess(abstract)

  corpus[paperId] = {'title': processed_title, 'abstract': processed_abstract}

def construct_positional_indexes(corpus: dict) -> dict:
    """
    Constructs positional indexes for the processed data by inserting words into a trie and creating positional indexes and posting lists.

    Parameters
    ----------
    corpus : dict
        The processed data.

    Returns
    -------
    dict
        The constructed positional indexes.
    """
    positional_index = defaultdict(lambda: defaultdict(lambda: {'title': [], 'abstract': []}))
    # TODO: Insert words from the processed data into a trie and construct positional indexes and posting lists.
    for ـid, document in corpus.items():

        for position, token in enumerate(document['title']):
            positional_index[token][ـid]['title'].append(position)

        for position, token in enumerate(document['abstract']):
            positional_index[token][ـid]['abstract'].append(position)
    return positional_index

docs = construct_positional_indexes(corpus)
print(len(docs))


34780


<div dir='rtl'>
<h1>2.
مدل‌های برداری و احتمالاتی
</h1>
<p>
در این بخش قصد داریم تا با استفاده از مدل‌های برداری و احتمالاتی، دو سیستم بازیابی اطلاعات طراحی کنیم. در نهایت قرار است سیستم‌های طراحی شده در این بخش را مورد ارزیابی قرار دهیم.

در مدل‌های برداری ما به ازای هر داک که در اختیار داریم و کوئری ورودی یک بردار در فضا در نظر می‌گیریم. در ادامه برای بدست آوردن میزان ارتباط داک‌ها و کوئری، از معیارهایی مانند ضرب داخلی بردارها استفاده می‌کنیم. در این بخش ابتدا بردارهای مربوط به هرکدام را با استفاده از مقادیر
tf, idf
می‌سازیم و سپس به سراغ محاسبه امتیاز داک‌ها می‌رویم تا در نهایت بتوانیم با استفاده از این امتیازها داک‌هایی مرتبط‌تر را خروجی دهیم و یک سیستم بازیابی کامل را طراحی کنیم.

در مدل‌های احتمالاتی اساس کار محاسبه
$P(R | d, q)$
است. در این روش می‌خواهیم به نوعی داک‌ها را به صورت مدل‌های احتمالاتی ببینیم. در درس مشاهده کردید که در انتها مدل‌های احتمالاتی هم رفتاری مشابه با مدل‌های برداری دارند. در ادامه یک سیستم مبتنی بر این مدل‌ها و با روش
Okapi25
طراحی می‌کنید.
</p>
</div>

<div dir='rtl'>
<h2>2-1.
ساخت ماتریس TF-IDF
</h2>
<p>
در این مرحله دو تابع
<code dir="ltr">get_tf(token, doc_id)</code>
و
<code dir="ltr">get_idf(token)</code>
را پیاده‌سازی کنید که تابع اول مقدار
tf
توکن ورودی را در شناسه‌ی موردنظر
و تابع دوم مقدار
idf
توکن ورودی را برروی نمایه‌ی کنونی حساب می‌کند و آن را به عنوان خروجی برمی‌گرداند.
</p>
</div>

$$tf_{t, d} = \text{Numbers of } t \text{ in the title of document } d + \text{Numbers of } t \text{ in the abstract of document } d$$
$$idf_t = \log\Biggl(\frac{\text{Number of documents}}{\text{Number of documents that contains } t + 1}\Biggr)$$

In [11]:
def convert(single_item_list):
    if len(single_item_list) == 1:
        return single_item_list[0]
    else:
        return None

In [12]:
def get_tf(token: str, positional_index: dict, doc_id: str) -> int:
    """
    Retrieves the term frequency (TF) of a given token within a specific document.

    Parameters
    ----------
    token : str
        The token for which to retrieve the term frequency.
    positional_index : dict
        A positional index that maps tokens to their positions in documents.
    doc_id : str
        The unique identifier of the document in which to calculate the TF.

    Returns
    -------
    int
        The term frequency (TF) of the given token within the specified document.
    """

    # TODO: Implement the calculation of term frequency (TF) for the token in the specified document.
    tf_title = 0
    tf_abstract = 0

    if token in positional_index:
        if doc_id in positional_index[token]:
            tf_title = len(positional_index[token][doc_id].get('title', []))
            tf_abstract = len(positional_index[token][doc_id].get('abstract', []))

    return tf_title + tf_abstract
token = convert(preprocessor.preprocess('Atmospheric'))
get_tf(token, docs, '1875fe689a5438d4e9561c218a4cf377ac901b82')

1

In [13]:
def get_idf(token: str,positional_index: dict) -> float:
    """
    Retrieves the inverse document frequency (IDF) of a given token.

    Parameters
    ----------
    token : str
        The token to retrieve the IDF for.

    Returns
    -------
    float
        The IDF of the given token.
    """

    # TODO: Implement the calculation of IDF for the token based on your document collection.
    if token not in positional_index:
        return 0

    unique_docs = set()

    for doc_id in positional_index[token]:
        unique_docs.add(doc_id)

    all_docs = len(corpus.keys())
    df = len(unique_docs)
    idf = math.log10(all_docs / (df + 1))
    return idf

t = convert(preprocessor.preprocess('consid'))
get_idf(t ,docs)

1.2397492485260002

<div dir='rtl'>
<p>
در مرحله‌ی نهایی تابع
<code dir="ltr">generate_tfidf_list(corpus, positional_index)</code>
را پیاده‌سازی کنید که با ورودی گرفتن
corpus
و
positional_index
مقادیر
tf-idf
را برای هر
token
به دست می‌آورد و به شکل یک ماتریس به عنوان خروجی برمی‌گرداند.
</p>
</div>

In [14]:

def generate_tfidf_list(corpus: dict, positional_index: dict) -> list[dict[str, float]]:
    """
    Generates a list of dictionaries representing documents with associated TF-IDF scores.

    Parameters
    ----------
    corpus : dict
        The corpus containing the processed data.
    positional_index : dict
        The positional index containing the term frequencies and document frequencies.

    Returns
    -------
    list[dict[str, float]]
        The list of dictionaries representing documents with associated TF-IDF scores.
    """

    # TODO: Write the function to generate the list of dictionaries with TF-IDF scores.
    tf_idf_list = []
    all_doc = corpus.items()

    for _id, doc in corpus.items():
        tf_idf_doc = {}
        tokens = doc['title'] + doc['abstract']

        for t in set(tokens):
            t = convert(preprocessor.preprocess(t))
            tf = get_tf(t, docs, _id)
            idf = get_idf(t ,docs)
            tf_idf_doc[t] = tf * idf

        tf_idf_list.append(tf_idf_doc)
    return tf_idf_list


tfidf = generate_tfidf_list(corpus, docs)

for tfidf_doc in tfidf[:15]:
    print(tfidf_doc)

{'ultrason': 2.9461012064846184, 'analysi': 0.8057727724158736, 'sic': 3.3140779917792127, 'vibrat': 2.645071210820637, 'amplitud': 2.393259237826838, 'grind': 3.0922292421628566, 'silicon': 2.5870792638429507, 'carbid': 3.490169250834894, 'surfac': 1.5870792638429505, 'ceram': 2.615107987443194, 'face': 1.893572155208434, 'ground': 2.2229975224318803, 'minor': 2.2597203294566204}
{'hiv1': 2.6772558941920384, 'use': 0.7882573930789332, 'chimera': 2.9461012064846184, 'robust': 1.280654236292263, 'activ': 0.9231428846758338, 'creat': 1.6513201600976388, 'need': 1.203488281479964, 'pseudotyp': 6.378278510341826, 'replicationcompet': 3.490169250834894, 'virussarscov2': 3.490169250834894, 'chimer': 2.8881092595069315, 'report': 1.202367520904668, 'monoclon': 2.328801248599919, 'assay': 1.6328367544036257, 'plasma': 1.8617803207845824, 'repr': 0, 'emerg': 1.287408563441694, 'ha': 0.7051955408804934, 'virusba': 0, 'antibodi': 5.114518247472382, 'viru': 0.0, 'sarscov2': 2.9846923401783374, 'po

In [15]:
def show_document_vector(vector: list[dict[str, float]], doc_index: int) -> None:
    """
    Prints the non-zero weights and corresponding terms for a document represented as a vector.

    Parameters
    ----------
    vector : list of dict of str:float
        The list of document vectors, where each document vector is a dictionary with term weights.
    doc_id : int
        The document index for which to display the vector.

    Returns
    -------
    None
    """
    vector_list = [(term, weight) for term, weight in vector[doc_index].items() if weight > 0]
    vector_list.sort(key=lambda x: x[1], reverse=True)
    length = math.sqrt(sum(weight ** 2 for _, weight in vector_list))
    normalized = {term: tfidf / length for term, tfidf in vector_list}
    for term, tfidf in vector_list:
        print(f'{term.ljust(16)}\t{tfidf}\t(normalized: {normalized[term]})')

show_document_vector(tfidf, 0)

carbid          	3.490169250834894	(normalized: 0.3797625948820716)
sic             	3.3140779917792127	(normalized: 0.36060224228340043)
grind           	3.0922292421628566	(normalized: 0.3364630528141514)
ultrason        	2.9461012064846184	(normalized: 0.3205629751887145)
vibrat          	2.645071210820637	(normalized: 0.287808136075011)
ceram           	2.615107987443194	(normalized: 0.28454786110177666)
silicon         	2.5870792638429507	(normalized: 0.28149807754096096)
amplitud        	2.393259237826838	(normalized: 0.2604086716325276)
minor           	2.2597203294566204	(normalized: 0.24587840713367512)
ground          	2.2229975224318803	(normalized: 0.24188262713425737)
face            	1.893572155208434	(normalized: 0.2060381097811723)
surfac          	1.5870792638429505	(normalized: 0.17268885724563357)
analysi         	0.8057727724158736	(normalized: 0.08767550710177546)


<div dir='rtl'>
<h2>2-2.
امتیازدهی به سندها
</h2>
<p>
در این بخش می‌خواهیم تا با سه شیوه‌ی متفاوت، داکیومنت‌ها را امتیازدهی کرده و kتا با بالاترین امتیاز را برگردانیم.
روش‌ها به شکل زیر است:

- **ltc.lnc:** در این روش از cosine similarity استفاده می‌کنیم و داده‌ها را بر اساس آن مرتّب می‌کنیم. برای متوجّه شدن نحوه‌ی عملکرد این سیستم، می‌توانید به اسلایدهای درس مراجعه کنید. همچنین دقت کنید که لازم است این امتیازدهی را هم به روش
term-at-a-time
و هم به روش
doc-at-a-time
محاسبه کنید.

- **Okapi25:** برای پیاده‌سازی این روش از تساوی زیر استفاده کنید:
</p>
</div>
<div>
<p>

$$ RSV_d = \sum_{t \in q} idf(t)\times\frac{(k_1 + 1)tf(t, d)}{k_1((1 - b) + b\frac{dl(d)}{avg(dl)}) + tf(t, d)} $$
</p>
</div>
<div dir='rtl'>
<p>
برای محاسبه این دو معیار، از توابعی که در بخش قبل پیاده‌سازی کردید استفاده کنید.

در تابع search که مربوط به جست و جوی پرسمان کلی است، به عنوان ورودی پارامتر پرسمان (query)، روش محاسبه امتیاز (method)، تعداد اسنادی که باید برگردانده شود (max-result-count) را ورودی می گیرید. ورودی وزن
(weight)
نشان می‌دهد که امتیاز نهایی چه وزنی از امتیاز عنوان و چکیده دارد. به زبان دیگر:
</p>
</div>
<div>
<p>

$$ final\_score = weight \times title\_score + (1 - weight) \times abstract\_score $$
</p>
</div>
<div dir='rtl'>
<p>

**توجّه کنید** که تابع نوشته شده، صرفاً یک prototype است و تا وقتی که نیازمندی‌های گفته شده را پیاده کنید، تغییر تابع مشکلی ندارد.
</p>
</div>

In [16]:
def get_df_sec(token, positional_index, section):
    return sum(1 for index in positional_index.get(token, {}).values() if section in index and index[section])

def get_idf_sec(token, positional_index, total_docs , section):
    df = get_df_sec(token, positional_index , section)
    if df == 0:
        return 0
    return math.log10(total_docs / df)

In [17]:
def cosine_norm(tfidf_vector):
    norm = np.sqrt(sum(tfidf ** 2 for tfidf in tfidf_vector.values()))
    return {term: tfidf / norm for term, tfidf in tfidf_vector.items()} if norm else tfidf_vector


def cosine_similarity(vec1, vec2):
    common_terms = set(vec1.keys()).intersection(set(vec2.keys()))
    dot_product = sum(vec1[term] * vec2[term] for term in common_terms)
    norm1 = np.sqrt(sum(value ** 2 for value in vec1.values()))
    norm2 = np.sqrt(sum(value ** 2 for value in vec2.values()))
    if norm1 > 0 and norm2 > 0:
        return dot_product / (norm1 * norm2)
    else:
        return 0.0


In [18]:
from collections import Counter
from collections import defaultdict
all_idf = {}
all_tf = defaultdict(lambda: defaultdict(dict))
positional_index = construct_positional_indexes(corpus)
N = len(corpus)
for token in positional_index.keys():
    for section in ['title', 'abstract']:
        df = sum(1 for doc_info in positional_index.get(token, {}).values() if section in doc_info and doc_info[section])
        all_idf[(token, section)] = math.log10(N / df) if df else 0

for token, docs_info in positional_index.items():
    for doc_id, positions_info in docs_info.items():
        tf_title = len(positions_info.get('title', []))
        tf_abstract = len(positions_info.get('abstract', []))
        all_tf[doc_id][token]['title'] = tf_title
        all_tf[doc_id][token]['abstract'] = tf_abstract



In [19]:
def get_tfidf_vector_for_doc(doc_id, field, N, doc):
    tfidf_vector = {}
    for term in doc:
            idf = all_idf[(term, field)]
            tf = all_tf[doc_id][term][field]
            tf = 1 + math.log(tf) if tf > 0 else 0
            tfidf_vector.setdefault(term, 0)
            tfidf_vector[term] = tf * idf
    return cosine_norm(tfidf_vector)

    
def get_tfidf_vector_for_query(query):
    tfidf_vector = defaultdict(int)
    for word in query:
        tfidf_vector[word] += 1
    result = {}
    for word, tf in tfidf_vector.items():
        result[word] = 1 + math.log10(tf)
    return result

In [20]:

def search(query: str, max_result_count: int, method: str = 'ltc-lnc-tat', weight: float = 0.5):
    """
    Finds relevant documents to query

    Parameters
    ---------------------------------------------------------------------------------------------------
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
                        notice that if max_result_count = -1, then you have to return all docs

    method: 'ltc-lnc-tat' or 'ltc-lnc-dat' or 'okapi25'

    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retreived documents with snippet
    """

    tokens = []
    for t in query.split():
      processed_token = preprocessor.preprocess(t)
      tokens.append(convert(processed_token))

    process_query = preprocessor.preprocess(query)

    score_title = {}
    for _id in list(corpus.keys()):
        score_title[_id] = 0

    score_abstract = {}
    for _id in list(corpus.keys()):
        score_abstract[_id] = 0

    # BM25 score calculation
    k1 = 1.2
    b = 0.6

    tf_title = 0
    tf_abstract = 0
    
    final_scores = defaultdict(float)
    N = len(corpus)

    if method == 'okapi25':
      total_length = sum(len(doc['title']) + len(doc['abstract']) for doc in corpus.values())
      avgdl = total_length / N
      for token in tokens:
          idf = get_idf(token, positional_index)
          for doc_id in corpus.keys():
              tf_title = len(positional_index.get(token, {}).get(doc_id, {}).get('title', []))
              tf_abstract = len(positional_index.get(token, {}).get(doc_id, {}).get('abstract', []))
              dl = len(corpus[doc_id]['title']) + len(corpus[doc_id]['abstract'])
              title_score = get_idf_sec(token, positional_index, N , 'title') * (tf_title * (k1 + 1)) / (tf_title + k1 * (1 - b + b * (dl / avgdl)))
              abstract_score = get_idf_sec(token, positional_index, N , 'abstract') * (tf_abstract * (k1 + 1)) / (tf_abstract + k1 * (1 - b + b * (dl / avgdl)))
              final_scores[doc_id] += weight * title_score + (1 - weight) * abstract_score

    elif method == 'ltc-lnc-dat':
      N = len(corpus)
      query_vector = get_tfidf_vector_for_query(process_query)
      for doc_id , doc in corpus.items():
          doc_vector_title = get_tfidf_vector_for_doc(doc_id, 'title', N, doc['title'])
          doc_vector_abstract = get_tfidf_vector_for_doc(doc_id, 'abstract', N,  doc['abstract'])
          title_score = cosine_similarity(query_vector, doc_vector_title)
          abstract_score = cosine_similarity(query_vector, doc_vector_abstract)
          final_scores[doc_id] += weight * title_score + (1 - weight) * abstract_score

    elif method == 'ltc-lnc-tat':
        query_vector = get_tfidf_vector_for_query(process_query)
        for word in query_vector:
            for doc_id, doc in corpus.items():
                if word in doc['title'] or word in doc['abstract']:
                    tf_title = all_tf[doc_id][word].get('title', 0)
                    tf_abstract = all_tf[doc_id][word].get('abstract', 0)
                    score_title = (1 + math.log10(tf_title)) * all_idf[(word, 'title')] if tf_title else 0
                    score_abstract = (1 + math.log10(tf_abstract)) * all_idf[(word, 'abstract')] if tf_abstract else 0
                    final_scores[doc_id] += query_vector[word] * (score_title * weight + score_abstract * (1 - weight))


    sorted_docs = sorted(final_scores.items(), key=lambda item: item[1], reverse=True)

    if max_result_count == -1:
        return [doc_id for doc_id, score in sorted_docs]
    else:
        return [doc_id for doc_id, score in sorted_docs[:max_result_count]]

<div dir='rtl'>
<h1>3.
مدل‌های زبانی
</h1>
<p>
در این بخش یک سیستم مبتنی بر مدل‌های زبانی پیاده‌سازی می‌کنید. مدل زبانی‌ای که باید استفاده کنید مدل
unigram
است که می‌توان گفت ساده‌ترین مدل ممکن است. در این مدل ما برای هر داک یک مدل دظر نظر می‌گیریم و احتمال حضور کلمات در مدل یک داک
نسبت به هم مستقل هستند. در پیاده‌سازی این بخش دستتان خیلی باز است. مدل‌هایی که تشکیل می‌دهید و نحوه
smoothing
و همچنین پارامترهای مدل همگی به خودتان برمی‌گردد.
</p>
</div>

In [21]:
def lm_search(query: str, max_result_count: int, mu = 100):
    """
    Finds relevant documents to query using Dirichlet smoothing.

    Parameters
    ---------------------------------------------------------------------------------------------------
    query: Query string.
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
    corpus: Dictionary of documents.
    tokens: Dictionary with total count of each token in the corpus.
    mu: Dirichlet prior parameter.

    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retrieved documents with snippet
    """
    prep = Preprocessor(None)
    query_tokens = prep.preprocess(query)

    tokens = {}
    for paperId, content in corpus.items():
        for token in content['title'] + content['abstract']:
            tokens[token] = tokens.get(token, 0) + 1

    lengthOfTokens = sum(tokens.values())

    scores = {}
    for doc_id, doc in corpus.items():
        doc_text = doc['title'] + doc['abstract']
        doc_length = len(doc_text)
        doc_score = {}

        for token in query_tokens:
            if token not in tokens.keys():
                continue
            tf = get_tf(token, docs, doc_id)
            # Dirichlet smoothing
            doc_score[token] = (tf + mu * (tokens[token] / lengthOfTokens)) / (doc_length + mu)
        if doc_score:
            scores[doc_id] = np.prod(list(doc_score.values()))

    sorted_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)
    result_docs = sorted_docs if max_result_count == -1 else sorted_docs[:max_result_count]
    return [doc_id for doc_id, _ in result_docs]


<div dir='rtl'>
<h1>4.
ارزیابی عملکرد سامانه
</h1>
<p>
در این بخش معیارهای زیر را برای سیستم‌های طراحی شده پیاده‌سازی کنید. سپس در بخش آخر با فراخوانی توابع پیاده‌سازی شده و ورودی دادن مقادیر موجود در فایل
validation.json
به عنوان ورودی
actual
توابع، می‌توانید معیارها را برای سیستم‌های بازیابی خودتان بدست بیاورید.
</p>
</div>

In [22]:
class Evaluation:
    def __init__(self, actual, predicts, queries):
        self.actual = actual
        self.queries = queries
        self.predicts = predicts

    def evaluate(self, require_scores):
        """
        Prints require scores for actual and predicts array

        Parameters
        ----------
        require_scores : List[str]
            Scores required to calculated

        Returns
        -------
        """
        methods = {
            'precision': self.precision,
            'recall': self.recall,
            'f1': self.f1,
            'map': self.map,
            'ndcg': self.ndcg,
            'mrr': self.mrr
        }

        for score in require_scores:
            if score in methods:
                print(f"{score}: {methods[score]()}")
            else:
                print("Scoring Function Not Found!")

    def precision(self):
        """
        Calculates the precision of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The precision of the predicted results
        """
        precision = 0.0
        num_queries = len(self.queries)
        for query in self.queries:
            actual = self.actual[query]
            predicted = self.predicts[query]

            tp = len(set(actual).intersection(set(predicted)))
            if len(predicted) > 0:
                query_precision = tp / len(predicted)
            else:
                query_precision = 0.0
            precision += query_precision

        if num_queries > 0:
            avg = precision / num_queries
        else:
            avg = 0.0

        return avg

    def recall(self):
        """
        Calculates the recall of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The recall of the predicted results
        """
        recall = 0.0
        num_queries = len(self.queries)

        for query in self.queries:
            actual = self.actual[query]
            predicted = self.predicts[query]

            tp = len(set(actual).intersection(set(predicted)))

            if len(actual) > 0:
                query_recall = tp / len(actual)
            else:
                query_recall = 0.0

            recall += query_recall

        if num_queries > 0:
            avg_recall = recall / num_queries
        else:
            avg_recall = 0.0

        return avg_recall

    def f1(self):
        """
        Calculates the F1 score of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The F1 score of the predicted results
        """
        if self.precision() + self.recall() == 0:
          return 0.0

        f1_score = 2 * self.precision() * self.recall() / (self.precision() + self.recall())
        return f1_score

    def map(self):
        """
        Calculates the Mean Average Precision of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The Mean Average Precision of the predicted results
        """
        map_score = 0.0

        for q in self.queries:
          actual = set(self.actual[q])
          predicted = self.predicts[q]
          precisions = []

          for _id in actual:
              if _id in predicted:
                  index = predicted.index(_id)
                  precision_at_k = len(set(predicted[:index + 1]).intersection(actual)) / (index + 1)
                  precisions.append(precision_at_k)

          if precisions:
              map_score += sum(precisions) / len(precisions)

        map = map_score / len(self.queries) if self.queries else 0.0
        return map

    def ndcg(self):
        """
        Calculates the Normalized Discounted Cumulative Gain (NDCG) of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The NDCG of the predicted results
        """
        total_ndcg = 0.0

        for query in self.queries:
          actual = self.actual[query]
          predicted = self.predicts[query]

          relevance = {doc_id: rank for rank, doc_id in enumerate(reversed(actual), 1)}

          dcg, idcg = 0.0, 0.0
          for i, doc_id in enumerate(predicted):
              if doc_id in relevance:
                  dcg += relevance[doc_id] / math.log2(i + 2)

          for rank, doc_id in enumerate(actual, 1):
              idcg += rank / math.log2(rank + 1)
          ndcg = dcg / idcg if idcg > 0 else 0
          total_ndcg += ndcg

        mean_ndcg = total_ndcg / len(self.queries) if self.queries else 0.0 
        return mean_ndcg

    def mrr(self):
        """
        Calculates the Mean Reciprocal Rank of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The MRR of the predicted results
        """
        rr = 0.0

        for query in self.queries:
            actual_set = set(self.actual[query])
            reciprocal_rank = 0
            for i, pred in enumerate(self.predicts[query]):
                if pred in actual_set:
                    reciprocal_rank = 1 / (i + 1)
                    break
            rr += reciprocal_rank

        mrr_score = (rr / len(self.actual)) if len(self.actual) > 0 else 0.0

        return mrr_score


In [23]:
import json
require_scores = ["precision", "recall", "f1", "map", "ndcg", "mrr"]

with open('./AI & Bioinformatics/validation.json', 'r') as f:
    validation_data = json.load(f)

actuals = {query: q_acts for query, q_acts in validation_data.items()}
queries = list(validation_data.keys())

In [24]:
predicts = {query: search(query, 10, method='ltc-lnc-dat') for query in queries}

eval_vsm = Evaluation(actuals, predicts, queries)
eval_vsm.evaluate(require_scores)

precision: 0.6
recall: 0.6
f1: 0.6
map: 0.8374090608465609
ndcg: 0.9597103186837869
mrr: 1.0


In [25]:
predicts = {query: search(query, 10, method='ltc-lnc-tat') for query in queries}

eval_vsm = Evaluation(actuals, predicts, queries)
eval_vsm.evaluate(require_scores)

precision: 0.6333333333333333
recall: 0.6333333333333333
f1: 0.6333333333333333
map: 0.7301596487360377
ndcg: 0.9100692813713037
mrr: 0.7321428571428571


In [26]:
predicts =  {query: search(query, 10, method='okapi25') for query in queries}

eval_bm = Evaluation(actuals, predicts, queries)
eval_bm.evaluate(require_scores)

precision: 0.6833333333333332
recall: 0.6833333333333332
f1: 0.6833333333333332
map: 0.7227412184009406
ndcg: 0.9433533304432862
mrr: 0.7555555555555555


In [27]:
predicts = {query: lm_search(query, 10) for query in queries}

eval_lm = Evaluation(actuals, predicts, queries)
eval_lm.evaluate(require_scores)

precision: 0.6833333333333332
recall: 0.6833333333333332
f1: 0.6833333333333332
map: 0.703412514697237
ndcg: 0.92315169505595
mrr: 0.7238095238095239
